In [7]:
#rawdata preprocess
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

def normalization(data):
    _range = np.max(data,axis=0) - np.min(data,axis=0)
    return (data-np.min(data,axis=0)) / _range

rawdata = pd.read_csv(r'/Users/jianghua/Desktop/hj_internship/data/ordered_data.csv')
lowVB = rawdata[rawdata.ToolWear <= 100]
lowVB_X = lowVB.iloc[:,:-1].values
lowVB_y = lowVB.iloc[:,-1].values
highVB = rawdata[rawdata.ToolWear>100]
highVB_X = highVB.iloc[:,:-1].values
highVB_y = highVB.iloc[:,-1].values

LOWVB_RANGE = np.max(lowVB_y) - np.min(lowVB_y)
LOWVB_MIN = np.min(lowVB_y)
HIGHVB_RANGE = np.max(highVB_y) - np.min(highVB_y)
HIGHVB_MIN = np.min(highVB_y)

lowVB_X = normalization(lowVB_X)
lowVB_y = normalization(lowVB_y)
highVB_X = normalization(highVB_X)
highVB_y = normalization(highVB_y)

lowVB_Xtrain, lowVB_Xtest, lowVB_ytrain, lowVB_ytest = train_test_split(lowVB_X,lowVB_y,test_size=0.2,random_state=1)
highVB_Xtrain,highVB_Xtest,highVB_ytrain,highVB_ytest=train_test_split(highVB_X,highVB_y,test_size=0.2,random_state=1)
lowVB_Xtrain = lowVB_Xtrain[:,1:]
lowVB_ytrain = lowVB_ytrain.reshape(len(lowVB_ytrain),1)
lowVB_Xmean = np.mean(lowVB_Xtrain,axis=0)
lowVB_ymean = np.mean(lowVB_ytrain,axis=0)
lowVB_Xtrain = lowVB_Xtrain - lowVB_Xmean
lowVB_ytrain = lowVB_ytrain - lowVB_ymean
lowVB_Xtest = lowVB_Xtest[:,1:]
lowVB_ytest = lowVB_ytest.reshape(len(lowVB_ytest),1)
lowVB_Xtest = lowVB_Xtest - lowVB_Xmean

highVB_Xtrain = highVB_Xtrain[:,1:]
highVB_ytrain = highVB_ytrain.reshape(len(highVB_ytrain),1)
highVB_Xmean = np.mean(highVB_Xtrain,axis=0)
highVB_ymean = np.mean(highVB_ytrain,axis=0)
highVB_Xtrain = highVB_Xtrain - highVB_Xmean
highVB_ytrain = highVB_ytrain - highVB_ymean
highVB_Xtest = highVB_Xtest[:,1:]
highVB_ytest = highVB_ytest.reshape(len(highVB_ytest),1)
highVB_Xtest = highVB_Xtest - highVB_Xmean


In [8]:
import torch
import torch.utils.data as Data
import torch.nn.functional as F
from torch.autograd import Variable
from sklearn import metrics

#mse_list=[]
LR = 0.01
BATCH_SIZE = [5,6,7]
EPOCH = [100,150,200]
NEURONS1 = [6,7,8,9,10]
NEURONS2 = [0,6,7,8,9,10]
HYPER_PARAMETERS = []
for i in NEURONS2:
    for j in NEURONS1:
        for m in BATCH_SIZE:
            for n in EPOCH:
                HYPER_PARAMETERS.append([i,j,m,n])


torch_lowVB_Xtrain = torch.from_numpy(lowVB_Xtrain)
torch_lowVB_ytrain = torch.from_numpy(lowVB_ytrain)
torch_lowVB_Xtest = torch.from_numpy(lowVB_Xtest)
lowVB_ytest_original = lowVB_ytest*LOWVB_RANGE+LOWVB_MIN

torch_highVB_Xtrain = torch.from_numpy(highVB_Xtrain)
torch_highVB_ytrain = torch.from_numpy(highVB_ytrain)
torch_highVB_Xtest = torch.from_numpy(highVB_Xtest)
highVB_ytest_original = highVB_ytest*HIGHVB_RANGE+HIGHVB_MIN

torch.manual_seed(1)
torch_lowVB_dataset = Data.TensorDataset(torch_lowVB_Xtrain,torch_lowVB_ytrain)
torch_highVB_dataset = Data.TensorDataset(torch_highVB_Xtrain,torch_highVB_ytrain)

lowVB_flag = 1
lowVB_MseValues = []
lowVB_MseValuesStr = []
highVB_MseValues = []
highVB_MseValuesStr = []

for torch_dataset in (torch_lowVB_dataset,torch_highVB_dataset):
    for hyperparameters in HYPER_PARAMETERS:
        loader = Data.DataLoader(dataset=torch_dataset,
                                 batch_size=hyperparameters[2],
                                 shuffle=True,
                                num_workers=2)


        if hyperparameters[0] == 0:
            net = torch.nn.Sequential(
                        torch.nn.Linear(12,hyperparameters[1]),
                        torch.nn.Dropout(0.5),
                        torch.nn.ReLU(),
                        torch.nn.Linear(hyperparameters[1],1)
                    )
        else:
            net = torch.nn.Sequential(
                        torch.nn.Linear(12,hyperparameters[1]),
                        torch.nn.Dropout(0.5),
                        torch.nn.ReLU(),
                        torch.nn.Linear(hyperparameters[1],hyperparameters[0]),
                        torch.nn.Dropout(0.5),
                        torch.nn.ReLU(),
                        torch.nn.Linear(hyperparameters[0],1)
                    )        
        opt_SGD = torch.optim.SGD(net.parameters(),lr=LR)
        loss_func = torch.nn.MSELoss()

        #losses_his = []
        for epoch in range(hyperparameters[3]):
            for step, (b_x,b_y) in enumerate(loader):
                output = net(b_x.float())
                loss = loss_func(output,b_y.float())
                opt_SGD.zero_grad()
                loss.backward()
                opt_SGD.step()
                #losses_his.append(loss)
        net.eval()
        if lowVB_flag == 1:
            ypred = net(torch_lowVB_Xtest.float())
            ypred = (ypred.data.numpy()+lowVB_ymean) * LOWVB_RANGE + LOWVB_MIN
            mse = metrics.mean_squared_error(lowVB_ytest_original,ypred)
            lowVB_MseValues.append(mse)
            mse_str = 'Neurons2_{}-Neuron1_{}-BatchSize_{}-Epoch_{}-Mse_{:.2f}'.format(
            hyperparameters[0],hyperparameters[1],hyperparameters[2],
            hyperparameters[3],mse
            )
            lowVB_MseValuesStr.append(mse_str)
        else:
            ypred = net(torch_highVB_Xtest.float())
            ypred = (ypred.data.numpy()+highVB_ymean) * HIGHVB_RANGE + HIGHVB_MIN
            mse = metrics.mean_squared_error(highVB_ytest_original,ypred)
            highVB_MseValues.append(mse)
            mse_str = 'Neurons2_{}-Neuron1_{}-BatchSize_{}-Epoch_{}-Mse_{:.2f}'.format(
            hyperparameters[0],hyperparameters[1],hyperparameters[2],
            hyperparameters[3],mse
            )
            highVB_MseValuesStr.append(mse_str)            
        net.train()
    lowVB_flag = 0

In [9]:
#find the best neural network
print(lowVB_MseValuesStr[np.argmin(lowVB_MseValues)])
print(highVB_MseValuesStr[np.argmin(highVB_MseValues)])

Neurons2_0-Neuron1_8-BatchSize_6-Epoch_150-Mse_903.92
Neurons2_0-Neuron1_9-BatchSize_5-Epoch_200-Mse_262.44


In [10]:
#save the best model
_LR = 0.01
_HYPER_PARAMETERS = [[8,6,150],[9,5,200]]
_lowVB_flag = 1

for torch_dataset in (torch_lowVB_dataset,torch_highVB_dataset):
    if _lowVB_flag == 1:
        hyperparameters = _HYPER_PARAMETERS[0]
    else:
        hyperparameters = _HYPER_PARAMETERS[1]
        
    loader = Data.DataLoader(
        dataset = torch_dataset,
        batch_size = hyperparameters[1], 
        shuffle = True,
        num_workers=2
    )
    best_nn = torch.nn.Sequential(
        torch.nn.Linear(12,hyperparameters[0]),
        torch.nn.Dropout(0.5),
        torch.nn.ReLU(),
        torch.nn.Linear(hyperparameters[0],1)
    )
    _opt_SGD = torch.optim.SGD(best_nn.parameters(),lr=_LR)
    _loss_func = torch.nn.MSELoss()

    for epoch in range(hyperparameters[2]):
        for step, (b_x,b_y) in enumerate(loader):
            output = best_nn(b_x.float())
            loss = _loss_func(output,b_y.float())
            _opt_SGD.zero_grad()
            loss.backward()
            _opt_SGD.step()
    if _lowVB_flag == 1:
        best_nn.eval()
        ypred = net(torch_lowVB_Xtest.float())
        ypred = (ypred.data.numpy()+lowVB_ymean) * LOWVB_RANGE + LOWVB_MIN
        mse = metrics.mean_squared_error(lowVB_ytest_original,ypred)  
        print('LowVBMse',mse)
        torch.save(best_nn.state_dict(),'/Users/jianghua/Desktop/hj_internship/lowVB_bestNN.pkl')
    else:
        ypred = net(torch_highVB_Xtest.float())
        ypred = (ypred.data.numpy()+highVB_ymean) * HIGHVB_RANGE + HIGHVB_MIN
        mse = metrics.mean_squared_error(highVB_ytest_original,ypred)
        print('HighVBMse',mse)
        torch.save(best_nn.state_dict(),'/Users/jianghua/Desktop/hj_internship/highVB_bestNN.pkl')
    _lowVB_flag = 0
        

LowVBMse 1026.671761554113
HighVBMse 392.28428642790516
